任务描述：

从当当网搜索页面，按照关键词搜索，使用Python编写爬虫，自动爬取搜索结果中图书的书名、出版社、价格、作者和图书简介等信息。

当当搜索页面：http://search.dangdang.com/

单页面图书信息下载

Python中的 requests 库能够自动帮助我们构造向服务器请求资源的request对象，返回服务器资源的response对象。如果仅仅需要返回HTML页面内容，直接调用response的text属性即可。在下面的代码中，我们首先导入requests库，定义当当网的搜索页面的网址，设置搜索关键词为"机器学习"。然后使用 requests.get 方法获取网页内容。最后将网页的前1000个字符打印显示。

In [6]:
import requests #1. 导入requests 库
test_url = 'http://search.dangdang.com/?key='+ '机器学习'  #2. 设置网页的URL地址
content_page = requests.get(test_url).text #3. 执行页面请求，返回页面内容
print(content_page[:1000]) #4.将页面的前1000个字符打印显示出来

    <!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
    <html xmlns="http://www.w3.org/1999/xhtml" xmlns:v="urn:schemas-microsoft-com:vml">
        <head>
                <base href="//search.dangdang.com/Standard/Search/Extend/hosts/" />
<link rel="dns-prefetch" href="//search.dangdang.com">
<link rel="dns-prefetch" href="//img4.ddimg.cn">
<title>机器学习-当当网</title>
<meta http-equiv="Content-Type" content="text/html; charset=GB2312">
<meta name="description" content="当当网在线销售机器学习等商品，并为您购买机器学习等商品提供品牌、价格、图片、评论、促销等选购信息" />
<meta name="keywords" content="机器学习" />
<meta name='ddclick_ab' content="ver:429"/>
<meta name="ddclick_search" content="key:机器学习|cat:|session_id:1940287ce9878d9990b2a258921b1c74|ab_ver:G|qinfo:88379_1_60|pinfo:_1_60"/>
<link rel="canonical" href="//search.dangdang.com/?key=%E6%9C%BA%E5%99%A8%E5%AD%A6%E4%B9%A0"/>
        <link rel="stylesheet" type="text/css" href="css/theme_1.css"/>
  


图书内容解析

下面开始做页面的解析，分析源码.使用Chrome浏览器直接打开网址 http://search.dangdang.com/?key=机器学习 。然后选中任意一本图书信息，鼠标右键点击“检查”按钮。不难发现搜索结果中的每一个图书的信息在页面中为\< li \>标签

我们以书名信息的提取为例进行具体说明。点击 li 标签下的 class属性为 name 的 p 标签，我们发现书名信息保存在一个name属性取值为"itemlist-title"的 a 标签的title属性中

我们可以使用xpath直接描述上述定位信息为 //li/p/a[@name="itemlist-title"]/@title 。下面我们用 lxml 模块来提取页面中的书名信息。利用etree.HTML，将字符串转化为Element对象,Element对象具有xpath的方法,返回结果的列表，xpath的使用请参考 https://www.w3school.com.cn/xpath/xpath_syntax.asp 。

In [14]:
from lxml import etree #导入etree模块
page = etree.HTML(content_page) #将页面字符串解析成树结构
book_name = page.xpath('//li/p/a[@name="itemlist-title"]/@title') #用xpath提取出书名信息。
book_name[:10] #打印提取出的前10个书名信息

[' 机器学习 机器学习*修订版！人工智能领域中文的开山之作，大牛周志华教授带你全面揭开机器学习的奥秘。',
 ' 机器学习公式详解 人工智能领域中文的开山之作、周志华 西瓜书 机器学习 伴侣书，Datawhale开源协作学习笔记 南瓜书，机器学习初学小白提升数学基础能力的练习书！',
 ' 机器学习实战：基于Scikit-Learn Keras和TensorFlow（原书第2版） 机器学习畅销书全新升级，基于TensorFlow 2和Scikit-Learn新版本，详细讲解构建智能系统所需要的概念和工具',
 ' 零基础学机器学习 人工智能教程籍，深入浅出神经网络与深度学习入门，基于python框架算法，机器学习入门小白书，与小冰一起课堂培训丰富的实战案例赠送全书实例源代码、案例数据集。',
 ' 深度学习 人工智能算法，机器学习奠基之作，AI圣经 AI圣经 deep learning中文版 2018年图灵奖获奖者作品 业内人称“花书” 人工智能机器学习深度学习领域奠基性经典畅销书',
 ' 机器学习实战【python基础教程指南，python核心编程实例指导，对wxpython数据库充分的讲解，不可错过的编程 【图灵程序设计丛书】人工智能开发图书 深度学习实践应用 利用Python阐述机器学习算法 配合日常用例 强劲实战导向 程序员人手常备',
 ' 机器学习算法竞赛实战 kaggle、阿里天池、广告算法竞赛入门 人工智能系统书籍 ',
 ' Python机器学习基础教程 基于Python3的机器学习入门教程 以机器学习算法实践为重点 使用scikit-learn库从头构建机器学习应用 涵盖模型评估调参方法 交差验证网格搜索 管道概念及文本数据处理方法等内容',
 ' 机器学习入门 一看就会，零基础打开机器学习入门',
 ' 深入机器学习 概念及原理讲解深入浅出、形象生动，数学原理及公式推导过程详细、大专可懂，包含算法完整实现，可真正上手实战']

下面我们编写一个函数 extract_books_from_content，输入一个页面内容，自动提取出页面包含的所有图书信息

In [18]:
from lxml import etree
def extract_books_from_content(content_page):
    books = []
    page = etree.HTML(content_page)
    book_name = page.xpath('//li/p/a[@name="itemlist-title"]/@title') #书名
    pub_info = page.xpath('//li/p[@class="search_book_author"]')#出版信息
    pub_info = [book_pub.xpath('string(.)') for book_pub in pub_info]
    price_now = page.xpath('//li//span[@class="search_now_price"]/text()')#当前价格
    stars = page.xpath('//li/p[@class="search_star_line"]/span[@class="search_star_black"]/span/@style') #星级
    comment_num = page.xpath('//li/p[@class="search_star_line"]/a[@class="search_comment_num"]/text()') #评论数
    for book in zip(book_name, pub_info, price_now, stars, comment_num):
        books.append(list(book))
    return books
books = extract_books_from_content(content_page)
books[:5]

[[' 机器学习 机器学习*修订版！人工智能领域中文的开山之作，大牛周志华教授带你全面揭开机器学习的奥秘。',
  '周志华 /2016-01-01  /清华大学出版社',
  '¥75.60',
  'width: 90%;',
  '11701条评论'],
 [' 机器学习公式详解 人工智能领域中文的开山之作、周志华 西瓜书 机器学习 伴侣书，Datawhale开源协作学习笔记 南瓜书，机器学习初学小白提升数学基础能力的练习书！',
  '谢文睿，秦州 /2022-07-01  /人民邮电出版社',
  '¥29.90',
  'width: 100%;',
  '8029条评论'],
 [' 机器学习实战：基于Scikit-Learn Keras和TensorFlow（原书第2版） 机器学习畅销书全新升级，基于TensorFlow 2和Scikit-Learn新版本，详细讲解构建智能系统所需要的概念和工具',
  '奥雷利安 /2022-12-27  /机械工业出版社',
  '¥117.70',
  'width: 90%;',
  '7217条评论'],
 [' 零基础学机器学习 人工智能教程籍，深入浅出神经网络与深度学习入门，基于python框架算法，机器学习入门小白书，与小冰一起课堂培训丰富的实战案例赠送全书实例源代码、案例数据集。',
  '黄佳 /2020-12-01  /人民邮电出版社',
  '¥67.30',
  'width: 100%;',
  '1769条评论'],
 [' 深度学习 人工智能算法，机器学习奠基之作，AI圣经 AI圣经 deep learning中文版 2018年图灵奖获奖者作品 业内人称“花书” 人工智能机器学习深度学习领域奠基性经典畅销书',
  '[美]Ian Goodfellow（伊恩・古德费洛）、[ /2017-07-01  /人民邮电出版社',
  '¥84.00',
  'width: 80%;',
  '72364条评论']]

为了显示的方便，我们将上述提取到的图书信息转换成 Pandas 的 DataFrame 格式。

In [21]:
import pandas as pd
books_df = pd.DataFrame(data=books,columns=["书名","出版信息","当前价格","星级","评论数"])
books_df[:10]

,书名,出版信息,当前价格,星级,评论数
0,包邮 机器学习的数学理论|8071561,[中] 史斌（Bin Shi） [美] S.S.艾扬格（S.S.Iyengar /2020-...,¥48.30,width: 0%;,1条评论
1,【正版特价】R语言机器学习：实用案例分析|230337,（印度）拉格哈夫・巴利（Raghav Bali） （印度）迪潘简・撒卡尔（Dip /2017...,¥28.32,width: 0%;,1条评论
2,包邮 机器学习入门：数学原理解析及算法实践|8088570,董政 /2022-04-20 /机械工业出版社,¥55.30,width: 0%;,7条评论
3,包邮 机器学习：算法视角（原书第2版）|8058334,新西兰] 史蒂芬？马斯兰（Stephen Marsland） /2019-04-01 /机...,¥69.30,width: 60%;,105条评论
4,构建实时机器学习系统 Amazon、微软、Google资深机器学习专家倾力打造，融合10年...,彭河森 汪涵 /2017-08-25 /机械工业出版社,¥51.10,width: 0%;,1条评论
5,包邮 机器学习：基于约束的方法|8071715,[意大利]马可・戈里（Marco Gori） /2020-08-07 /机械工业出版社,¥83.30,width: 100%;,9条评论
6,包邮 机器学习开发实战（英文版）|8078304,[意]迪诺・埃斯波西托（Dino Esposito） 弗朗西斯科・埃斯波西托（F /2021...,¥97.30,width: 0%;,3条评论
7,包邮 机器学习实战：基于Scikit-Learn、Keras和TensorFlo|8075...,[法] 奥雷利安・杰龙（Aurélien Géron） /2020-09-23 /机械工业出版社,¥104.30,width: 0%;,2条评论
8,包邮 [套装书]机器学习实战+Python深度学习实战+TensorFlow机器|8061...,[法] 奥雷利安・杰龙（Aurélien Géron）[印] 纳温・库马尔・马纳 /2018...,¥188.36,width: 100%;,77条评论
9,场景化机器学习 人工智能入门教程机器学习实战神经网络编程AI书籍深度学习框架实战方法基础教程书,道格・哈金 等 /2021-01-01 /人民邮电出版社,¥61.60,width: 100%;,95条评论


In [22]:
books_df.shape

(35, 5)

多页面图书信息下载

观察搜索页面最底部，输入一个关键词，通常会返回多页结果，点击任意一个页面按钮，然后观察浏览器地址栏的变化。我们发现不同页面通过浏览器URL中添加 page_index 属性即可。例如我们搜索"机器学习"关键词，访问第10页结果，则使用以下URL:

http://search.dangdang.com/?key=机器学习&page_index=10

假设我们一共希望下载10页内容，则可以通过以下代码实现。

In [23]:
import time
key_word = "机器学习" #设置搜索关键词
max_page = 10 #需要下载的页数
books_total = []
for page in range(1,max_page+1):
    url = 'http://search.dangdang.com/?key=' + key_word + "&page_index=" + str(page) #构造URL地址
    page_content = requests.get(url).text #下载网页内容
    books = extract_books_from_content(page_content) #网页图书信息解析
    books_total.extend(books) #将当前页面的图书信息添加到结果列表
    print("page " + str(page) +", "+ str(len(books)) + " books downloaded." )
    time.sleep(1) #停顿1秒再下载下一页

page 1, 46 books downloaded.
page 2, 48 books downloaded.
page 3, 53 books downloaded.
page 4, 54 books downloaded.
page 5, 55 books downloaded.
page 6, 56 books downloaded.
page 7, 55 books downloaded.
page 8, 52 books downloaded.
page 9, 36 books downloaded.
page 10, 35 books downloaded.


In [24]:
books_total_df = pd.DataFrame(data=books_total, columns=["书名","出版信息","当前价格","星级","评论数"])

In [25]:
books_total_df.sample(5)

,书名,出版信息,当前价格,星级,评论数
183,【正版特价】Python大规模机器学习|232383,[荷]巴斯蒂安・贾丁（Bastiaan Sjardin） [意]卢卡・马萨罗（L /2019...,¥22.25,width: 100%;,1178条评论
289,机器学习提升法 理论与算法 机器学习理论，人工智能导论，机器学习导论和算法图解，提升法创始...,[美]罗伯特・夏皮雷（Robert E. Schapire）约夫・弗雷德 (Yo /2020...,¥86.10,width: 0%;,1条评论
217,机器学习的算法观点 第2版 理论与实践完美结合的机器学习经典入门教材，包括了机器学习的各种...,[新西兰] 史蒂芬・马斯兰 (Stephen Marsland) /2022-08-01 ...,¥96.70,width: 0%;,7条评论
392,应用机器学习方法度量在线品牌忠诚度模型构建研究 思源文库,赵玮 /2017-07-01 /北京交通大学出版社,¥47.20,width: 100%;,30条评论
95,机器学习在量化金融中的应用 金融数据分析专家多年工作结晶，深入浅出阐释机器学习数学基础及其...,倪好 于光希 郑劲松 董欣 /2021-02-01 /清华大学出版社,¥62.50,width: 60%;,267条评论


In [26]:
books_total_df.shape

(490, 5)

In [27]:
books_total_df.to_csv("books_total.csv",encoding="utf8",sep="\t",index=None)